<a href="https://colab.research.google.com/github/ailab-nda/ML/blob/main/Demucs_demo_mp3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demucs による音源分離

論文<br>
https://arxiv.org/abs/2111.03600<br>
<br>
GitHub<br>
https://github.com/facebookresearch/demucs<br>
<br>

## ライブラリのインストール

In [ ]:
!pip install demucs
!pip install yt-dlp moviepy
!pip install pydub
!npm install -g deno

## ライブラリのインポート

In [ ]:
import demucs.separate
from yt_dlp import YoutubeDL
from moviepy.editor import VideoFileClip, AudioFileClip
import IPython

# テスト動画のセットアップ

In [ ]:
video_url = 'https://www.youtube.com/watch?v=8ZP5eqm4JqM' #@param {type:"string"}

## ダウンロード

In [ ]:
download_resolution = 360
input_clip_path = 'input_clip.mp4'
input_audio_path = 'input_clip.mp3'

# 動画ダウンロード --> input_clip.mp4
ydl_opts = {'format': f'best[height<={download_resolution}]', 'overwrites': True, 'outtmpl': input_clip_path}
with YoutubeDL(ydl_opts) as ydl:
    ydl.download([video_url])

# 音声だけ取り出し --> input_clip.mp3
!yes|ffmpeg -i input_clip.mp4 input_clip.mp3

### 確認（ビデオ）

In [ ]:
# mp4動画の再生
from IPython.display import HTML
from base64 import b64encode

mp4 = open('input_clip.mp4', 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="80%" height="80%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

### 確認（音声）

In [ ]:
IPython.display.Audio('input_clip.mp3')

# 音源を分離

In [ ]:
demucs.separate.main(["--mp3", input_audio_path])

## パートごとの再生

### 歌

In [ ]:
IPython.display.Audio('separated/htdemucs/input_clip/vocals.mp3')

### ドラム

In [ ]:
IPython.display.Audio('separated/htdemucs/input_clip/drums.mp3')

### ベース

In [ ]:
IPython.display.Audio('separated/htdemucs/input_clip/bass.mp3')

### その他

In [ ]:
IPython.display.Audio('separated/htdemucs/input_clip/other.mp3')

## カラオケ動画を作成

### 歌以外の抽出

In [ ]:
#!python3 -m demucs --mp3 --two-stems=vocals {input_audio_path}
demucs.separate.main(["--mp3", "--two-stems", "vocals", input_audio_path])

### 確認

In [ ]:
IPython.display.Audio('separated/htdemucs/input_clip/no_vocals.mp3')

### 映像と分離された音源との合成

In [ ]:
karaoke_video = 'karaoke.mp4'

videoclip = VideoFileClip('input_clip.mp4')
audioclip = AudioFileClip('separated/htdemucs/input_clip/no_vocals.mp3')

karaokeclip = videoclip.set_audio(audioclip)
karaokeclip.write_videofile(karaoke_video, codec='libx264', audio_codec='aac', temp_audiofile='temp-audio.m4a', remove_temp=True)

### 再生

In [ ]:
# mp4動画の再生
from IPython.display import HTML
from base64 import b64encode

mp4 = open('karaoke.mp4', 'rb').read()
data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
HTML(f"""
<video width="80%" height="80%" controls>
      <source src="{data_url}" type="video/mp4">
</video>""")

ファイルのダウンロード

In [ ]:
# ファイルをDL
from google.colab import files
import shutil
# フォルダをzip圧縮
shutil.make_archive("musicvideo", "zip", "/content/separated/htdemucs/input_clip")
# 圧縮ファイルをダウンロード
files.download("musicvideo.zip")
files.download("karaoke.mp4")